

## The conservative form

Many works in the early days of computational fluid dynamics in the 1960s showed that using the conservation form of the Euler equations is more accurate for situations with shock waves.  And as you already saw, the shock-tube solutions do contain shocks.

The conserved variables $\underline{\mathbf{u}}$ for Euler's equations are

$$
\begin{equation}
\underline{\mathbf{u}} = \left[
\begin{array}{c}
\rho \\
\rho u \\
\rho e_T \\ 
\end{array}
\right]
\end{equation}
$$

where $\rho$ is the density of the fluid, $u$ is the velocity of the fluid and $e_T = e + \frac{u^2}{2}$ is the specific total energy; $\underline{\mathbf{f}}$ is the flux vector:

$$
\begin{equation}
\underline{\mathbf{f}} = \left[
\begin{array}{c}
\rho u \\
\rho u^2 + p \\
(\rho e_T + p) u \\
\end{array}
\right]
\end{equation}
$$

where $p$ is the pressure of the fluid.

If we put together the conserved variables and the flux vector into our PDE, we get the following set of equations:

$$
\begin{equation}
    \frac{\partial}{\partial t}
    \left[
        \begin{array}{c}
            \rho \\
            \rho u \\
            \rho e_T \\
        \end{array}
    \right] +
    \frac{\partial}{\partial x}
    \left[
        \begin{array}{c}
            \rho u \\
            \rho u^2 + p \\
            (\rho e_T + p) u \\
        \end{array}
    \right] =
    0
\end{equation}
$$

There's one major problem there.  We have 3 equations and 4 unknowns.  But there is a solution!  We can use an equation of state to calculate the pressure—in this case, we'll use the ideal gas law.

## Calculating the pressure

For an ideal gas, the equation of state is

$$
e = e(\rho, p) = \frac{p}{(\gamma -1) \rho}
$$

where $\gamma = 1.4$ is a reasonable value to model air, 

$$
\therefore p = (\gamma -1)\rho e
$$ 

Recall from above that

$$
e_T = e+\frac{1}{2} u^2
$$

$$
\therefore e = e_T - \frac{1}{2}u^2
$$

Putting it all together, we arrive at an equation for the pressure

$$
p = (\gamma -1)\left(\rho e_T - \frac{\rho u^2}{2}\right)
$$

## Flux in terms of $\underline{\mathbf{u}}$

With the traffic model, the flux was a function of traffic density.  For the Euler equations, the three equations we have are coupled and the flux *vector* is a function of $\underline{\mathbf{u}}$, the vector of conserved variables:

$$
\underline{\mathbf{f}} = f(\underline{\mathbf{u}})
$$

In order to get everything squared away, we need to represent $\underline{\mathbf{f}}$ in terms of $\underline{\mathbf{u}}$.
We can introduce a little shorthand for the $\underline{\mathbf{u}}$ and $\underline{\mathbf{f}}$ vectors and define:

$$
\underline{\mathbf{u}} =
\left[
    \begin{array}{c}
        u_1 \\
        u_2 \\
        u_3 \\
    \end{array}
\right] =
\left[
    \begin{array}{c}
        \rho \\
        \rho u \\
        \rho e_T \\
    \end{array}
\right]
$$

$$
\underline{\mathbf{f}} =
\left[
    \begin{array}{c}
        f_1 \\
        f_2 \\
        f_3 \\
    \end{array}
\right] =
\left[
    \begin{array}{c}
        \rho u \\
        \rho u^2 + p \\
        (\rho e_T + p) u \\
    \end{array}
\right]
$$  

With a little algebraic trickery, we can represent the pressure vector using quantities from the $\underline{\mathbf{u}}$ vector.

$$
p = (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1} \right)
$$

Now that pressure can be represented in terms of $\underline{\mathbf{u}}$, the rest of $\underline{\mathbf{f}}$ isn't too difficult to resolve:

$$\underline{\mathbf{f}} = \left[ \begin{array}{c}
f_1 \\
f_2 \\
f_3 \\ \end{array} \right] =
\left[ \begin{array}{c}
u_2\\
\frac{u^2_2}{u_1} + (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1} \right) \\
\left(u_3 + (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1}\right) \right) \frac{u_2}{u_1}\\ \end{array}
\right]$$


In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
def u(rho,u):
    '''
    computes vector u
    parameters
    ----------
    rho: density
    u: velocity
    
    Returns
    --------
    u: vector
            [rho
            rho*u
            (rho*eT+P)*u]'''
    
    e=5.6636 #from table 
    gamma=1.4
    eT=e+0.5*u**2
    P=(gamma-1)*(rho*eT-(0.5*(rho*u**2)))
    u1=rho
    u2=rho*u**2+P
    u3=(rho*eT+P)*u
    u=np.array([u1,u2,u3])
    return u

In [3]:
def f(u):
    '''
    computes f given one value of u
    
    Parameters
    -----------
    u:  array of u, 
                    [rho
                    rho*u
                    (rho*eT+P)u]
    
    
    Returns
    -------
    f: flux array,
                    [rho*u
                    rho*u^2 +P
                    (rho*eT+P)u]
                    '''
    gamma=1.4
    f1=u[1]
    f2=((u[1]**2)/u[0])+(gamma-1)*(u[2]-0.5*((u[1]**2)/u[0]))
    f3=(u[2]+(gamma-1)*(u[2]-0.5*((u[1]**2)/u[0])))*(u[1]/u[0])
    f=np.array([f1,f2,f3])
    return f

In [4]:
def P(u):
    '''
    calculates the pressure
    
    Parameters
    ----------
    u: array of u
                [rho
                rho*u
                rho*eT]
    Returns
    --------
    P: pressure, float '''
    gamma=1.4
    P=(gamma-1)*(u[2]-0.5*((u[1]**2)/u[0]))
    return P